In [2]:

import sqlite3
from sqlite3 import Error
import os

In [4]:
import pandas as pd

AttributeError: module 'numpy.random' has no attribute 'Generator'

In [5]:

import sqlite3
from sqlite3 import Error
import os

def create_connection(db_file, delete_db=False):
    """Create or return a connection to a SQLite database."""
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    try:
        conn = sqlite3.connect(db_file)
        print("Connection established: Database is created at", db_file)
        return conn
    except Error as e:
        print(e)
        return None

def create_table(conn, create_table_sql):
    """Create a table from the create_table_sql statement."""
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        print("Table created successfully")
    except Error as e:
        print(e)

def load_data(conn, df, table_name):
    """Load data from a DataFrame into a specified table in the SQLite database."""
    try:
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Data loaded successfully into {table_name}")
    except Error as e:
        print(e)



In [6]:
db_file = 'normalise.db'
conn = create_connection(db_file)


Connection established: Database is created at normalise.db


In [7]:
def create_tables(normalized_conn):
    # Create Athletes table
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS Athletes (
        AthleteID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Gender TEXT NOT NULL,
        Age INTEGER
    );
    '''
    create_table(normalized_conn, create_table_sql)
    
    # Create Sports table
    
    create_table_sql = """
        CREATE TABLE IF NOT EXISTS Teams (
            TeamID INTEGER PRIMARY KEY,
            Team TEXT,
            NOC TEXT
        );
    """
    create_table(normalized_conn, create_table_sql)
    
    
    
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS Sports (
        SportID INTEGER PRIMARY KEY,
        Sport TEXT NOT NULL
    );
    '''
    create_table(normalized_conn, create_table_sql)

    
    # Create Venues table
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS Venues (
        VenueID INTEGER PRIMARY KEY,
        City TEXT NOT NULL
   );
    '''
    create_table(normalized_conn, create_table_sql)
    
    # Create Events table
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS Events (
        EventID INTEGER PRIMARY KEY,
        SportID INTEGER,
        VenueID INTEGER,
        Event TEXT NOT NULL,
        Year INTEGER,
        Season TEXT,
        FOREIGN KEY (SportID) REFERENCES Sports (SportID),
        FOREIGN KEY (VenueID) REFERENCES Venues (VenueID)
    );
    '''
    create_table(normalized_conn, create_table_sql)
    

    # Create Medals table
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS Medals (
        MedalID INTEGER PRIMARY KEY,
        MedalType TEXT NOT NULL
    );
    '''
    create_table(normalized_conn, create_table_sql)
    
    

   # Create AthleteEvents table
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS AthleteEvents (
    AthleteID INTEGER,
    EventID INTEGER,
    MedalID INTEGER,
    PRIMARY KEY (AthleteID,EventID,MedalID),
    FOREIGN KEY (AthleteID) REFERENCES Athletes(AthleteID),
    FOREIGN KEY (EventID) REFERENCES Events(EventID),
    FOREIGN KEY (MedalID) REFERENCES Medals(MedalID)
    );
    '''
    create_table(normalized_conn, create_table_sql)
                 
    

    # Create TeamEvents table
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS TeamEvents (
    TeamID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (TeamID, EventID),
    FOREIGN KEY (TeamID) REFERENCES Teams(TeamID),
    FOREIGN KEY (EventID) REFERENCES Events(EventID)
  );
    '''
    create_table(normalized_conn, create_table_sql)
    
    normalized_conn.commit()
    
create_tables(conn)   
    

Table created successfully
Table created successfully
Table created successfully
Table created successfully
Table created successfully
Table created successfully
Table created successfully
Table created successfully


In [7]:
athletes = data[['Name', 'Gender', 'Age']].drop_duplicates().reset_index(drop=True)
athletes['AthleteID'] = range(1, len(athletes) + 1)

In [ ]:
athletes.to_csv('Athlete.csv', index=False)

In [8]:
from IPython.display import FileLink

In [9]:
from IPython.display import FileLink
FileLink('output.csv')  # Creates a clickable link to download the file

C:\Users\Ranadeep Mahendra\DMQL Project\output.csv

In [10]:



team_data = data[['Team', 'NOC']].drop_duplicates().reset_index(drop=True)
team_data['TeamID'] = range(1, len(team_data) + 1)

sport_data = data[['Sport']].drop_duplicates().reset_index(drop=True)
sport_data['SportID'] = range(1, len(sport_data) + 1)

venue_data = data[['City']].drop_duplicates().reset_index(drop=True)
venue_data['VenueID'] = range(1, len(venue_data) + 1)

event_data = data[['Event', 'Sport', 'City', 'Year', 'Season']].drop_duplicates().reset_index(drop=True)
event_data['EventID'] = range(1, len(event_data) + 1)
event_data = pd.merge(event_data, sport_data[['Sport', 'SportID']], on='Sport', how='left')
event_data = pd.merge(event_data, venue_data[['City', 'VenueID']], on='City', how='left')


medal_data = data[data['Medal'].notna()]
unique_medals = medal_data['Medal'].drop_duplicates().reset_index(drop=True).reset_index()
unique_medals.rename(columns={'index': 'MedalID', 'Medal': 'MedalType'}, inplace=True)


# Merge to link MedalID
medal_data = pd.merge(medal_data, unique_medals, left_on='Medal', right_on='MedalType')

# Merge to link AthleteID and EventID
medal_data = pd.merge(medal_data, athletes, on=['Name', 'Gender', 'Age'])
medal_data = pd.merge(medal_data, event_data, on=['Event', 'Sport', 'Year', 'Season', 'City'])

# Select the relevant columns
athlete_events = medal_data[['AthleteID', 'EventID', 'MedalID']]

# Remove duplicates from the athlete_events DataFrame
athlete_events = athlete_events.drop_duplicates(subset=['AthleteID', 'EventID', 'MedalID'], keep='first')


team_events = pd.merge(data, team_data, on=['Team', 'NOC'])
team_events = pd.merge(team_events, event_data, on=['Event', 'Sport', 'Year', 'Season', 'City'])

# Selecting relevant columns for the TeamEvents table
team_events = team_events[['TeamID', 'EventID']].drop_duplicates()

NameError: name 'data' is not defined

In [9]:

athlete_events

,AthleteID,EventID,MedalID
0,4,4,0
1,26388,4,2
2,49004,4,2
3,101007,4,0
4,123872,4,2
...,...,...,...
42227,196312,5592,1
42228,174964,5592,2
42229,30692,5073,2
42230,129633,5239,2


In [8]:


load_data(conn, athletes, 'Athletes')
load_data(conn, team_data, 'Teams')
load_data(conn, sport_data, 'Sports')
load_data(conn, venue_data, 'Venues')
load_data(conn, event_data[['EventID', 'SportID','VenueID', 'Event', 'Year', 'Season']], 'Events')
load_data(conn,unique_medals,'Medals')
load_data(conn, athlete_events ,'AthleteEvents')
load_data(conn, team_events ,'TeamEvents')

#load_data(conn, team_events_data[['TeamID', 'EventID', 'Medal']], 'TeamEvents')

conn.commit()

print("All operations completed successfully, and all tables are loaded.")

Data loaded successfully into Athletes
Data loaded successfully into Teams
Data loaded successfully into Sports
Data loaded successfully into Venues
Data loaded successfully into Events
Data loaded successfully into Medals
Data loaded successfully into AthleteEvents
Data loaded successfully into TeamEvents
All operations completed successfully, and all tables are loaded.


In [13]:
team_data.to_csv('Teams.csv', index=False)

In [21]:
team_events.to_csv('TeamEvents.csv', index=False)

In [18]:
event_data[['EventID', 'SportID','VenueID', 'Event', 'Year', 'Season']].to_csv('Events.csv', index=False)